<a href="https://colab.research.google.com/github/anujkhaire/Machine-Learning/blob/main/Market_Basket_Analysis/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=211fea6441f2320facfbdda9e1d8be1b65ff2a25d2d4959f0519e96a10560a4a
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/colab_datasets/Market_Basket_Optimisation.csv',header=None)
#replacing empty value with 0.
df.fillna(0,inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,chutney,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,turkey,avocado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,mineral water,milk,energy bar,whole wheat rice,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
#for using aprori need to convert data in list format..
# transaction = [['apple','almonds'],['apple'],['banana','apple']]....
transactions = []
for i in range(0,len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0,20) if str(df.values[i,j])!='0'])

In [6]:
transactions[0]

['shrimp',
 'almonds',
 'avocado',
 'vegetables mix',
 'green grapes',
 'whole weat flour',
 'yams',
 'cottage cheese',
 'energy drink',
 'tomato juice',
 'low fat yogurt',
 'green tea',
 'honey',
 'salad',
 'mineral water',
 'salmon',
 'antioxydant juice',
 'frozen smoothie',
 'spinach',
 'olive oil']

In [11]:
  from apyori import apriori
  #Call apriori function which requires minimum support, confidance and lift, min length is combination of item default is 2".
  rules = apriori(transactions,min_support=0.003,min_confidance=0.2,min_lift=3,min_length=2)

In [8]:
rules

<generator object apriori at 0x7f1416c91e60>

In [9]:
# all rules need to be converted in a list..
Results = list(rules)

In [12]:
#convert result in a dataframe for further operation...
df_results = pd.DataFrame(Results)
df_results.head()

,items,support,ordered_statistics
0,"(cottage cheese, brownies)",0.003466,"[((brownies), (cottage cheese), 0.102766798418..."
1,"(chicken, light cream)",0.004533,"[((chicken), (light cream), 0.0755555555555555..."
2,"(escalope, mushroom cream sauce)",0.005733,"[((escalope), (mushroom cream sauce), 0.072268..."
3,"(escalope, pasta)",0.005866,"[((escalope), (pasta), 0.07394957983193277, 4...."
4,"(tomato juice, fresh bread)",0.004266,"[((fresh bread), (tomato juice), 0.09907120743..."


In [13]:
#keep support in a separate data frame so we can use later.. 
support = df_results.support

In [14]:
'''
convert orderstatistic in a proper format.
order statistic has lhs => rhs as well rhs => lhs we can choose any one for convience i choose first one which is 'df_results['ordered_statistics'][i][0]'
''' 

#all four empty list which will contain lhs, rhs, confidance and lift respectively.

first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list.. first and second element was frozenset which need to be converted in list..
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [15]:
#convert all four list into dataframe for further operation..
lhs = pd.DataFrame(first_values)
rhs= pd.DataFrame(second_values)
confidance=pd.DataFrame(third_values,columns=['Confidance'])
lift=pd.DataFrame(fourth_value,columns=['lift'])

In [22]:
#concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final

,0,1,0,1,2,support,Confidance,lift
0,brownies,None,cottage cheese,None,None,0.003466,0.102767,3.225330
1,chicken,None,light cream,None,None,0.004533,0.075556,4.843951
2,escalope,None,mushroom cream sauce,None,None,0.005733,0.072269,3.790833
3,escalope,None,pasta,None,None,0.005866,0.073950,4.700812
4,fresh bread,None,tomato juice,None,None,0.004266,0.099071,3.259356
...,...,...,...,...,...,...,...,...
89,ground beef,pancakes,mineral water,spaghetti,None,0.003066,0.211009,3.532991
90,ground beef,None,tomatoes,mineral water,spaghetti,0.003066,0.031208,3.344117
91,olive oil,None,mineral water,spaghetti,milk,0.003333,0.050607,3.216994
92,mineral water,milk,spaghetti,shrimp,None,0.003066,0.063889,3.014029


In [24]:
df_final.fillna(value=' ', inplace=True)
#set column name
df_final.columns = ['lhs',0,'rhs',1,2,'support','confidance','lift']
#add all three column because those where the lhs itemset only
df_final['lhs'] = df_final['lhs']+str(", ")+df_final[1]+str(", ")
df_final.head()

,lhs,0,rhs,1,2,support,confidance,lift
0,"brownies, , , ,",,cottage cheese,,,0.003466,0.102767,3.225330
1,"chicken, , , ,",,light cream,,,0.004533,0.075556,4.843951
2,"escalope, , , ,",,mushroom cream sauce,,,0.005733,0.072269,3.790833
3,"escalope, , , ,",,pasta,,,0.005866,0.073950,4.700812
4,"fresh bread, , , ,",,tomato juice,,,0.004266,0.099071,3.259356


In [18]:
#drop those 1,2 column because now we already appended to lhs column..
df_final.drop(columns=[0,1,2],inplace=True)

In [19]:
df_final.head()

,lhs,rhs,support,confidance,lift
0,"brownies, ,",cottage cheese,0.003466,0.102767,3.225330
1,"chicken, ,",light cream,0.004533,0.075556,4.843951
2,"escalope, ,",mushroom cream sauce,0.005733,0.072269,3.790833
3,"escalope, ,",pasta,0.005866,0.073950,4.700812
4,"fresh bread, ,",tomato juice,0.004266,0.099071,3.259356
